<a href="https://colab.research.google.com/github/murovadarya/-yatube_project/blob/main/BERT_for_russian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

import gzip
import shutil
import time
import pandas as pd
import requests
import torch
import torch.nn.functional as F
import torchtext
import transformers
from transformers import AutoTokenizer
from transformers import AutoModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


План:
1) делаем one-hot encoding
2) 

In [2]:
torch.backends.cudnn.deterministic = True
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
NUM_EPOCHS = 3

In [5]:
df = pd.read_csv('train_data_with_test.csv')

In [6]:

df.head(3)

,id,operator_id,thread_id,received_date,text,rank,Очередь,Тип,operator_text,user_text,text_all,len,first,Разметка
0,24879637,NaN,24879637,2021-12-01 08:24:43.627824+03,Что случилось: Магазин не принимает баллы Пров...,1,Акции,ticket,"Здравствуйте, Александр Александрович. Баллами...",Что случилось: Магазин не принимает баллы Пров...,Что случилось: Магазин не принимает баллы Пров...,2,Что случилось: Магазин не принимает баллы Пров...,Клиент интересуется где можно потратить баллы ...
1,24879977,NaN,24879977,2021-12-01 10:05:22.239252+03,Что случилось: Не приходят письма об операциях...,1,Акции,ticket,"Здравствуйте, Вера Кирилловна. Здорово, что вс...",Что случилось: Не приходят письма об операциях...,Что случилось: Не приходят письма об операциях...,3,Что случилось: Не приходят письма об операциях...,Пол-лю не приходят письма по операциям и новос...
2,24880438,NaN,24880438,2021-12-01 11:26:30.547176+03,Что случилось: Другая проблема Расскажите подр...,1,Акции,ticket,"Здравствуйте, Станислав Николаевич. Если за ме...",Что случилось: Другая проблема Расскажите подр...,Что случилось: Другая проблема Расскажите подр...,4,Что случилось: Другая проблема Расскажите подр...,NaN


**Label encoding**

In [7]:
possible_labels = df["Разметка"].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Клиент интересуется где можно потратить баллы или как их получать. Получает подробности по акциям': 0,
 'Пол-лю не приходят письма по операциям и новостям.': 1,
 nan: 2,
 'У клиента вопрос по кэшбэку партнера, нам нужны данные для поиска информации в отчете.': 3,
 'Клиент не получал кэшбэк по чеку из-за блокировки партнера, партнер снял блок.': 4,
 'У клиента списались рубли вместо баллов при отмене платежа с кэшбэком, так как баллы он уже потратил.': 5,
 'Клиенту не пришел кэшбэк от партнера,\xa0пересылаем ему его запрос ': 6,
 'Клиент думает, что не потратил баллы и заплатил только рублями.': 7,
 'Вопросы по бонусной цели - почему ее нет/почему с ней не могу потратить баллы/как установить или отключить': 8,
 'Клиент не получил кэшбэк из-за несоблюдения правил - исчерпал лимиит/ не вступил в лояльность/получил на другом кошельке/оплата в агрегатор': 9,
 'Клиент не получил кэшбэк из-за несоблюдения правил - исчерпал лимиит/ не вступил в лояльность/получил на другом кошельке/оплата в 

In [8]:
df['Encodings'] = df["Разметка"].replace(label_dict)
df.head(5)

,id,operator_id,thread_id,received_date,text,rank,Очередь,Тип,operator_text,user_text,text_all,len,first,Разметка,Encodings
0,24879637,NaN,24879637,2021-12-01 08:24:43.627824+03,Что случилось: Магазин не принимает баллы Пров...,1,Акции,ticket,"Здравствуйте, Александр Александрович. Баллами...",Что случилось: Магазин не принимает баллы Пров...,Что случилось: Магазин не принимает баллы Пров...,2,Что случилось: Магазин не принимает баллы Пров...,Клиент интересуется где можно потратить баллы ...,0
1,24879977,NaN,24879977,2021-12-01 10:05:22.239252+03,Что случилось: Не приходят письма об операциях...,1,Акции,ticket,"Здравствуйте, Вера Кирилловна. Здорово, что вс...",Что случилось: Не приходят письма об операциях...,Что случилось: Не приходят письма об операциях...,3,Что случилось: Не приходят письма об операциях...,Пол-лю не приходят письма по операциям и новос...,1
2,24880438,NaN,24880438,2021-12-01 11:26:30.547176+03,Что случилось: Другая проблема Расскажите подр...,1,Акции,ticket,"Здравствуйте, Станислав Николаевич. Если за ме...",Что случилось: Другая проблема Расскажите подр...,Что случилось: Другая проблема Расскажите подр...,4,Что случилось: Другая проблема Расскажите подр...,NaN,2
3,24880510,948.0,24880510,2021-12-01 11:38:12.385777+03,Вопрос: Где в истории операций или баллов увид...,1,Акции,ticket,Вопрос: Где в истории операций или баллов увид...,"Добрый день.Пожалуйста покажите скрином, а то ...",Вопрос: Где в истории операций или баллов увид...,10,Вопрос: Где в истории операций или баллов увид...,"У клиента вопрос по кэшбэку партнера, нам нужн...",3
4,24880808,110.0,24880808,2021-12-01 12:33:07.542175+03,Здравствуйте! Мы выбрали десять пользователей ...,1,Акции,ticket,Здравствуйте! Мы выбрали десять пользователей ...,"оооооооой, спасибо большое))))) ооочень приятн...",Здравствуйте! Мы выбрали десять пользователей ...,6,Здравствуйте! Мы выбрали десять пользователей ...,Клиент интересуется где можно потратить баллы ...,0


In [9]:
train_texts = df.iloc[:15000]['text_all'].values
train_labels = df.iloc[:15000]['Encodings'].values

valid_texts = df.iloc[15000:20000]['text_all'].values
valid_labels = df.iloc[15000:20000]['Encodings'].values

test_texts = df.iloc[20000:]['text_all'].values
test_labels = df.iloc[20000:]['Encodings'].values

In [10]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruBert-base")

Downloading:   0%|          | 0.00/590 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
valid_encodings = tokenizer(list(valid_texts), truncation=True, padding=True)

In [ ]:
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

In [ ]:
class TextsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
  
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx])
            for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = TextsDataset(train_encodings, train_labels)
valid_dataset = TextsDataset(valid_encodings, valid_labels)
test_dataset = TextsDataset(test_encodings, test_labels)

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=16, shuffle=True)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=16, shuffle=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=16, shuffle=False)

In [ ]:
model = AutoModel.from_pretrained("sberbank-ai/ruBert-base")
model.to(DEVICE)
model.train()

In [ ]:
optim = torch.optim.Adam(model.parameters(), lr=5e-5)

In [ ]:
from transformers import Trainer, TrainingArguments

In this code, we iterate over multiple epochs. In each epoch we perform the following steps:
1. Load the input into the device we are working on (GPU or CPU)
2. Compute the model output and loss
3. Adjust the weight parameters by backpropagating the loss
4. Evaluate the model performance on both the training and validation set

In [ ]:
!pip install datasets
from datasets import load_metric
import numpy as np


In [ ]:
metric = load_metric("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    # note: logits are a numpy array, not a pytorch tensor
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(
    predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(output_dir='./results',
                                  num_train_epochs=9,
                                  per_device_train_batch_size=6,
                                  per_device_eval_batch_size=6,
                                  logging_dir='./logs',
                                  logging_steps=10,)

trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_dataset,
                  eval_dataset=test_dataset,
                  compute_metrics=compute_metrics,
                  optimizers=(optim, None)) # optim and learning rate scheduler ... )


In [ ]:
trainer.train()

In [ ]:
print(trainer.evaluate())

In [ ]:
model.eval()
model.to(DEVICE)

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch", ...)